In [5]:
import requests
from bs4 import BeautifulSoup as b4
import pandas as pd
# Initialize lists
mobile_names = []
ratings = []
prices = []

for j in range(1,21):
    url = f"https://www.ebay.com/sch/i.html?_nkw=mobiles&_sacat=0&_from=R40&_pgn={j}"
    r = requests.get(url).content
    soup = b4(r, "html.parser")
    
    # Locate the product listing container
    table = soup.find("ul", class_="srp-results srp-list clearfix")
    List = table.find_all("li")
    
    # Loop through products, skip the 6th item
    for idx, i in enumerate(List):
        if idx == 5:
            continue  # Skip 6th item
    
        # Name
        img = i.find("img")
        if not img or not img.has_attr("alt"):
            break
        alt_text = img["alt"].strip()
    
        # Rating
        rating_div = i.find("div", class_="s-item__reviews")
        rating_span = rating_div.find("span", class_="clipped") if rating_div else None
        rating_text = rating_span.text.strip() if rating_span else "N/A"
    
        # Price
        price_div = i.find("div", class_="s-item__detail s-item__detail--primary")
        price_span = price_div.find("span", class_="s-item__price") if price_div else None
        price_text = next(price_span.stripped_strings, None) if price_span else None
    
        # Stop the loop if any essential field is missing
        if not alt_text or not price_text:
            break
    
        # Append only if all data is valid
        mobile_names.append(alt_text)
        ratings.append(rating_text)
        prices.append(price_text)

df=pd.DataFrame({"mobile_name":mobile_names,"ratings":ratings,"price":prices})
df.to_csv('mobile_info.csv')
df

,mobile_name,ratings,price
0,Samsung Galaxy A54 5G 128GB SM-A546 50 MP (SIM...,4.5 out of 5 stars.,$129.99
1,Tested work 100% Apple iPhone 5c - 8/16/32GB -...,4.0 out of 5 stars.,$34.00
2,Working well Apple iPhone 5 5c 5s 8/16/32/64GB...,4.0 out of 5 stars.,$28.88
3,TESLA EXPLR 9 256GB Smartphone | Rugged Phone ...,N/A,$225.00
4,Samsung Galaxy Note 7 FE (Unlocked)-64 GB-Vari...,N/A,$154.90
...,...,...,...
1435,Apple iPhone 14 Plus 128GB Unlocked AT&T T-Mob...,5.0 out of 5 stars.,$362.95
1436,Samsung Galaxy S21 Ultra 5G SM-G998U1 G998U Un...,4.5 out of 5 stars.,$275.99
1437,Apple iPhone 12 128GB - Fully Unlocked - GOOD ...,5.0 out of 5 stars.,$218.99
1438,CAT S62 - 128 GB - Black (T-Mobile),N/A,$311.22


In [ ]:
import requests
from bs4 import BeautifulSoup as b4
import pandas as pd
import time

mobile_names = []
ratings = []
no_reviews = []
bought_last_month = []
mobile_price = []

for j in range(1, 51):
    time.sleep(2)  # Pause to reduce chances of getting blocked

    url = f"https://www.amazon.com/s?k=phones&page={j}&xpid=JlnAWWhUg8SUy&crid=34IIB9Q6BYDU8&qid=1751122090&sprefix=%2Caps%2C359&ref=sr_pg_2"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                      "AppleWebKit/537.36 (KHTML, like Gecko) "
                      "Chrome/125.0.0.0 Safari/537.36",
        "Accept-Language": "en-US,en;q=0.9",
    }

    r = requests.get(url, headers=headers)
    
    # Check for CAPTCHA or block
    if "captcha" in r.text.lower() or "Enter the characters you see below" in r.text:
        print(f"[Blocked] CAPTCHA triggered on page {j}")
        continue

    soup = b4(r.text, "lxml")
    containers = soup.find_all("div", {"data-component-type": "s-search-result"})
    print(f"[Page {j}] Found {len(containers)} products")

    for c in containers:
        title = c.find("h2")
        mobile_names.append(title.text.strip() if title else "none")

        rating = c.find("a", class_="a-popover-trigger a-declarative")
        if rating and rating.get("aria-label"):
            r_rating = rating.get("aria-label")
            ratings.append(r_rating.strip()[0:18])
        else:
            ratings.append("none")

        number_of_reviews = c.find("span", class_="a-size-base s-underline-text")
        no_reviews.append(number_of_reviews.text.strip() if number_of_reviews else "none")

        last = c.find("span", class_="a-size-base a-color-secondary")
        bought_last_month.append(last.text.strip() if last else "none")

        price = c.find("span", class_="a-price")
        mobile_price.append(price.text.strip() if price else "none")

# Create DataFrame
df = pd.DataFrame({
    "Mobile Name": mobile_names,
    "Rating": ratings,
    "Number of Reviews": no_reviews,
    "Bought Last Month": bought_last_month,
    "Price": mobile_price
})

print(df)
df.to_csv("amazon_mobile_phones1.csv")

In [ ]:
#for some reason the price elements are repeated twice so this script solves it

import pandas as pd

# Load the CSV file
file_path = "amazon_mobile_phones1.csv"  # Update path if needed
df = pd.read_csv(file_path)

# Function to clean the price by keeping only up to the second '$' if present
def clean_price(price):
    if isinstance(price, str) and price.count("$") > 1:
        return price[:price.find("$", price.find("$") + 1)]
    return price

# Apply the function to the 'Price' column
df['Price'] = df['Price'].apply(clean_price)

# Optionally, save the cleaned data to a new CSV file
df.to_csv("cleaned_amazon_mobile_phones.csv", index=False)

# Display the first few cleaned rows
print(df[['Mobile Name', 'Price']])
